# Earth Lab Capstone Project: Where can soil moisture improve rainfall-triggered landslide predictability?

## Author: Jacquelyn Witte

## This Notebook examines the relationship between SMAP and ESA CCI soil moisture and GPM daily and IMERGE 30min precipitation for Landslides in the US

- Based on Landslide events from the NASA Global Landslide Catalog
- Using Landslide locations over Colorado as a workflow example

In [1]:
# Override saving plots
GLOBAL_CACHE_OVERRIDE = True

In [ ]:
import os
from glob import glob

import datetime as dt
import earthpy as et
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import scipy.stats as st
import seaborn as sns
import xarray as xr

import warnings
warnings.filterwarnings('ignore')

# Prettier plotting with seaborn
sns.set(font_scale=1.2)

In [ ]:
def read_smap(filepath, index):
    """
    Reads SMAP data and returns the variable of interest.
    
    Parameters
    ----------
    filepath: Str
        File path of a SMAP L3 HDF5 file
        
    group_id: String
        Groups within the file to access
        
    index: int
        Index associated with the variable to retrieve
    
    Returns
    -------
    data: 2D numpy.ndarray (lat, lon)
    date: Date String yyyymmdd
    """
    # Read the file
    group_id = 'Soil_Moisture_Retrieval_Data_PM'
    with h5py.File(filepath, 'r') as f:
        # Extract data info
        data_id = list(f[group_id].keys())[index]
        data = f[group_id][data_id][:,:]
        data[data == f[group_id][data_id].attrs['_FillValue']] = np.nan
        
        filename = os.path.basename(filepath)
        yyyymmdd= filename.split('_')[5]
        yyyy = int(yyyymmdd[0:4])
        mm = int(yyyymmdd[4:6])
        dd = int(yyyymmdd[6:8])
        date=dt.datetime(yyyy,mm,dd)
    return data, date

In [ ]:
def findfile(input_files, input_date):
    """
    Returns a single file from a list of files
    
    Parameters
    ----------
    input_files: List of strings
        List of full path to the file
        
    input_date: String
        YYYYMMDD format
        
    Returns
    -------
    file: Str
    """
    file = [x for x in input_files if re.findall(input_date, x)]
    if not file:
        raise ValueError('File does not exist for '+input_date)
    return file

In [ ]:
def nearestneighbor_ncdf(input_file, parameter, loc):
    """
    Extracts nearest neighbor value based on location and desired parameter. 
    
    Parameters
    ----------   
    input_file: Str - full path to a single file
    
    parameter: Str 
    
    loc: tuple (degree longtitude, degree latitude)
    
    Returns
    -------
    float
    """
    # read the netcdf file
    try:
        data_xr = xr.open_dataset(input_file).squeeze()
    except IOError:
        print("This file is not accessible: "+input_file)
    finally:
        data_xr.close()
    
    # subset the file
    res = data_xr[parameter].sel(indexers={
            'lon': loc[0],
            'lat': loc[1]},
            method="nearest")
    
    return float(res.values)

In [ ]:
def leastsq(x, y, method=3):
    """
    5 methods to compute least squares fit.

    Reference: https://github.com/sbird/spb_common/blob/master/leastsq.py
    Compute the least squares fit to y = beta x + alpha,
    using one of the 5 methods outlined in
    http://adsabs.harvard.edu/abs/1990ApJ...364..104I
    Method 1 minimises distance from Y given X (ie, the standard least squares fit)
    Method 2 minimises distance from X given Y
    Method 3 (recommended) is the OLS bisector, which gives a line bisecting the above two.
    Method 4 (Orthogonal regression) minimises perpendicular distance from the line to points
    Method 5 is the geometric mean of the slopes from methods 1 and 2.
    Method 6 is the Theil-Sen estimator: the median of the pairwise slopes.
    (See Akritas 95,  http://www.tandfonline.com/doi/abs/10.1080/01621459.1995.10476499)

    Parameters
    ----------
    x, y: numeric array

    Returns
    -------
    (alpha, beta, bvar), the intercept slope and variance of the slope
    """
    # Define some sums
    xbar = np.mean(x)
    ybar = np.mean(y)
    xdif = x-xbar
    ydif = y-ybar
    sxx = np.sum(xdif**2)
    syy = np.sum(ydif**2)
    sxy = np.sum(ydif*xdif)

    # Check for zeros
    if sxx == 0 or syy == 0 or sxy == 0:
        raise ValueError("Least Squares ill-defined")
    if method > 6 or method < 1:
        raise ValueError("Method not recognised")

    # These formulas are taken from Table 1 of Isobe et al, page 3
    # Minimise distance from Y given X
    beta1 = sxy/sxx
    # Variance of b1
    bvar1 = np.sum(xdif**2*(ydif-beta1*xdif)**2)/sxx**2
    # Minimise distance from X given Y
    beta2 = syy/sxy
    # Variance of b2
    bvar2 = np.sum(ydif**2*(ydif-beta2*xdif)**2)/sxy**2
    # Covariance of b1 and b2
    covb12 = np.sum(xdif*ydif*(ydif-beta2*xdif) *
                    (ydif-beta1*xdif))/(beta1*sxx**2)

    if method == 1:
        beta = beta1
        bvar = bvar1
    if method == 2:
        beta = beta2
        bvar = bvar2
    if method == 3:
        # OLS bisector: line that bisects the above two.
        beta1p1 = 1+beta1**2
        beta2p1 = 1+beta2**2
        beta = (beta1*beta2 - 1 + np.sqrt(beta1p1*beta2p1))/(beta1+beta2)
        # Variance
        prefac = beta**2 / ((beta1 + beta2)**2 * beta1p1 * beta2p1)
        var = (beta2p1**2 * bvar1 + 2 * beta1p1 * beta2p1 
               * covb12 + beta1p1**2 * bvar2)
        bvar = prefac*var

    if method == 4:
        # Orthogonal: minimise perpendicular distance from line to points
        beta = 0.5*((beta2-1./beta1)+np.sign(sxy) *
                    np.sqrt(4+(beta2-1./beta1)**2))
        prefac = beta**2 / (4*beta1**2 + (beta1*beta2 - 1)**2)
        bvar = prefac * (bvar1/beta1**2 + 2*covb12 + beta1**2*bvar2)

    if method == 5:
        # Reduced major axis:
        beta = np.sign(sxy)*np.sqrt(beta1*beta2)
        bvar = 0.25 * (beta2/beta1 * bvar1 + 2*covb12 + beta1/beta2 * bvar2)

    if method == 6:
        # Theil-Sen estimator for uncensored data: the median of the slopes.
        yy = np.subtract.outer(y, y)
        xx = np.subtract.outer(x, x)
        ind = np.where(xx != 0)
        beta = np.median(yy[ind]/xx[ind])
        # Can't find a formula for the variance
        bvar = 0

    # The intercept
    alpha = ybar - beta*xbar

    return (alpha, beta, bvar)

In [ ]:
def get_imerge_hires(imerge_files, glc_df):
    """
    Reads all IMERGE 30min CSV file into a dataFrame

    
    # Ref: https://www.geeksforgeeks.org/ways-to-filter-pandas-dataframe-by-column-values/

    Parameters
    ----------
    imerge_files: List of strings
        List of full path to the file
    
    glc_df: dataFrame
        Global Landslide Catalog 
    
    Returns
    -------
    imerge: dataFrame
        Contains datetime, landslide ID, precipitation

    """
    id_list = glc_df['event_id'].values.tolist()
    
    list = []
    for f in imerge_files:
        #temp_df = pd.read_csv(f).set_index('datetime')
        print(f)
        temp_df = pd.read_csv(f)
        # filter for landslide id
        list.append(temp_df[temp_df['id'].isin(id_list)])

    imerge = pd.concat(list)
    # convert datetime to pd datetime because some dates are not in the right format
    imerge['datetime'] = pd.to_datetime(imerge['datetime'])
    # Create a simple date string to compare with the GLC data
    imerge['yyyymmdd'] = pd.to_datetime(imerge['datetime']).dt.strftime('%Y%m%d')
    #imerge.index = pd.to_datetime(imerge.index)
    imerge = imerge.reset_index().set_index('datetime')
    return imerge

### Choose the state 

In [ ]:
westernUS = ['Colorado', 'California', 'Oregon', 
             'Washington','Utah', 'Idaho']
state = westernUS[0]

### Read and subset to Landslides >= year 2015 (SMAP data starts in 2015)
- Based on the state chosen

In [ ]:
# Read the GLC file
glc_file = os.path.join(et.io.HOME,
                        'earth-analytics',
                        'data', 'capstone', 
                        'landslide', 'nasa_global_landslide_catalog_point.csv')

glc = pd.read_csv(glc_file)

glc_state = glc[(glc['country_code'] == 'US') 
             & (glc['admin_division_name'] == state) 
             & (glc['landslide_trigger'] != 'freeze_thaw') 
             & (glc['landslide_trigger'] != 'snowfall_snowmelt') 
             & (glc['landslide_trigger'] != 'earthquake') 
             & (glc['landslide_trigger'] != 'leaking_pipe') 
             & (glc['landslide_trigger'] != 'no_apparent_trigger') 
             & (glc['landslide_trigger'] != 'other')              
             & (glc['landslide_trigger'] != 'unknown')
            ]
# convert to pandas datetime
glc_state['date'] = pd.to_datetime(glc_state['event_date'])
glc_state = glc_state.set_index('date').sort_index()
glc_state_gt2015 = glc_state[glc_state.index > '2015-04-01']

print(glc_state_gt2015.shape)
print(np.unique(glc_state_gt2015['landslide_trigger']))
print(np.unique(glc_state_gt2015['landslide_category']))
print(glc_state_gt2015.columns)

# Dropping last dataframe if state=Utah - SMAP data does not exist
if state == 'Utah':
    glc_state_gt2015 = glc_state_gt2015.drop(pd.to_datetime('2019-06-26 04:00:00'))

glc_state_gt2015.tail()

### Get all SMAP, ESA, GPM and IMERGE data files, sorted
 - ESA = Percent of Saturation Soil Moisture
 - SMAP = Volumetric soil moisture in cm3/cm3
 - GPM = Daily precipitation in mm
 - IMERGE = 30 min precipitation in mm

In [ ]:
data_dir = os.path.join(et.io.HOME,
                        'earth-analytics',
                        'data', 'capstone')
smap_files = sorted(glob(os.path.join(data_dir, 'smap_9km', '*.h5')))

# GPM daily files
gpm_files = sorted(glob(os.path.join(data_dir, 'gpm_westernUS', '*nc4')))

# GPM 30 min files
gpm_hires_files = sorted(glob(os.path.join(data_dir,
                                           'precip_imerge',
                                           'imerge',
                                           'glc', 'imerge*.csv')))

esa_files = sorted(glob(os.path.join(data_dir, 'esa_soil_moisture',
                                     '*ACTIVE*nc')))

# Print a sample as a sanity check
print(os.path.exists(smap_files[0]))
print(os.path.exists(gpm_files[0]))
print(os.path.exists(gpm_hires_files[0]))
print(os.path.exists(esa_files[0]))

### Load the EASE2 grid lon and lat datasets to subset SMAP data. 
- These can be found on the NSIDC website: https://nsidc.org/data/ease/tools#geo_data_files

> Brodzik, M. J., B. Billingsley, T. Haran, B. Raup, M. H. Savoie. 2012. EASE-Grid 2.0: Incremental but Significant Improvements for Earth-Gridded Data Sets. ISPRS International Journal of Geo-Information, 1(1):32-45, doi:10.3390/ijgi1010032. http://www.mdpi.com/2220-9964/1/1/32.

#### These are SMAP variables that can provide key information in characterizing landslides over Colorado

| Variable Name | Index | Units |
| ----------- | ----------- | ----------- |
| soil_moisture | 24 | cm<sup>3</sup>/cm<sup>3</sup> |
| radar_water_body_fraction | 15 | N/A | 
| vegetation_opacity | 46 | N/A |
| vegetation_water_content | 50 | kg/m<sup>2</sup> |

In [ ]:
# Reshape to dimensions of the SMAP data above
sm_data, date = read_smap(smap_files[1], 24)

lats = np.fromfile(os.path.join(data_dir, 'smap_9km',
                                'EASE2_M09km.lats.3856x1624x1.double'),
                   dtype=np.float64).reshape(sm_data.shape)
lons = np.fromfile(os.path.join(data_dir, 'smap_9km',
                                'EASE2_M09km.lons.3856x1624x1.double'),
                   dtype=np.float64).reshape(sm_data.shape)
sm_data.shape

### Read the SMAP, ESA CCI and GPM data 
- Two dataframes are generated
    1. Colocated to all the Landslide events
    2. Precipitation measurements going back 7 day from each landslide event and indexed to Landslide ID
        - GPM daily resolution precipitation

In [ ]:
precip7d_date = []
precip7d = []
precip_accum = []
precip_max = []
smap_sm = []
smap_sm_7d = []
smap_wc = []
esa_sm = []
esa_sm_7d = []
landslide_date = []
landslide_id = []
landslide7d_id = []
landslide_cat = []
landslide_trig = []
landslide_sz = []
lat = []
lon = []
periods = 7
count_down = []

for i, ls_date in enumerate(glc_state_gt2015.index):

    # Get -7 days from the event
    glc_date = pd.date_range(ls_date, periods=periods,
                             freq='-1D').strftime('%Y%m%d')

    # Append landslide metadata
    landslide_date.append(ls_date)
    lat.append(glc_state_gt2015.latitude[i])
    lon.append(glc_state_gt2015.longitude[i])
    landslide_id.append(glc_state_gt2015.event_id[i])
    landslide_cat.append(glc_state_gt2015.landslide_category[i])
    landslide_trig.append(glc_state_gt2015.landslide_trigger[i])
    landslide_sz.append(glc_state_gt2015.landslide_size[i])

    # Take the +/- 0.3 deg mean around the Landslide event
    N_lat = glc_state_gt2015.latitude[i]+0.15
    S_lat = glc_state_gt2015.latitude[i]-0.15
    W_lon = glc_state_gt2015.longitude[i]-0.15
    E_lon = glc_state_gt2015.longitude[i]+0.15
    subset = (lats < N_lat) & (lats > S_lat) & (lons > W_lon) & (lons < E_lon)

    #print(i, ls_date, glc_state_gt2015.latitude[i], glc_state_gt2015.longitude[i])
    
    # Initialize data
    sm_max = []
    vegwc_max = []
    esa_mean = []
    precip = []
    countd = periods
 
    # loop over the 7 days
    for yyyymmdd in glc_date:
        
        # Find the SMAP file
        filesm = findfile(smap_files, yyyymmdd)
        # Retrieve the SMAP variables
        sm, time_t = read_smap(filesm[0], 24)
        vegwc, t = read_smap(filesm[0], 50)
        # Calculate the SMAP max
        sm_max.append(np.nanmax(sm[subset]))
        vegwc_max.append(np.nanmax(vegwc[subset]))
        
        # Get the Landslide location
        loc = (glc_state_gt2015.longitude[i], glc_state_gt2015.latitude[i])

        # Find the ESA soil moisture file
        file_esa = findfile(esa_files, yyyymmdd) 
        # Get the nearest neighbor value of % soil moisture
        res_esa = nearestneighbor_ncdf(file_esa[0], 'sm', loc)
        # Replace negative values with NaN
        if res_esa < 0.0:
            esa_mean.append(np.nan)
        else:
            esa_mean.append(res_esa)
        
        # find the GPM file
        file_gpm = findfile(gpm_files, yyyymmdd) 
        
        precip7d.append(nearestneighbor_ncdf(file_gpm[0], 
                             'precipitationCal', loc))
        landslide7d_id.append(glc_state_gt2015.event_id[i])
        # Append the date
        precip7d_date.append(yyyymmdd)

        precip.append(nearestneighbor_ncdf(file_gpm[0], 
                             'precipitationCal', loc))
        
        # Append countdown
        count_down.append(countd)
        countd -= 1

    # Append the summary values for the 7 day period
    smap_sm.append(np.nanmax(sm_max))
    smap_wc.append(np.nanmax(vegwc_max))
    esa_sm.append(np.nanmax(esa_mean))
    smap_sm_7d.extend(sm_max)
    esa_sm_7d.extend(esa_mean)
    
    # Filter for low precipitation values
    if np.nansum(precip) < 0.4:
        precip_accum.append(np.nan)
        precip_max.append(np.nan) 
    else:
        precip_accum.append(np.nansum(precip))
        precip_max.append(np.nanmax(precip))
    
# Create a soils and precip dataFrame
landslide_df = pd.DataFrame(smap_sm,
                            index=pd.to_datetime(landslide_date), 
                            columns=['smap_sm'])
landslide_df['veg_water_content'] = smap_wc
landslide_df['esa_sm_percent'] = esa_sm
landslide_df['gpm_7day_accum_mm'] = precip_accum
landslide_df['gpm_7day_max_mm'] = precip_max

# Add the Landslide metadata
landslide_df['glc_lat'] = lat
landslide_df['glc_lon'] = lon
landslide_df['landslide_id'] = landslide_id
landslide_df['landslide_category'] = landslide_cat
landslide_df['landslide_trigger'] = landslide_trig
landslide_df['landslide_size'] = landslide_sz

# Create the 7day precipitation dataFrame
landslide_precip7d_df = pd.DataFrame(precip7d,
                            index=pd.to_datetime(precip7d_date), 
                            columns=['gpm_precip_mm'])
landslide_precip7d_df['landslide_id'] = landslide7d_id
landslide_precip7d_df['smap_sm']= smap_sm_7d
landslide_precip7d_df['esa_vol']= esa_sm_7d


# Add the cumulative precipitation
accum = []
for i, data in landslide_precip7d_df.groupby("landslide_id"):
    res = data['gpm_precip_mm'].cumsum().values
    if res.max() > 0.4:
        accum.extend(np.flip(data['gpm_precip_mm'].cumsum().values))
    else:
        accum.extend([np.nan]*periods)

landslide_precip7d_df['gpm_7day_accum_mm'] = accum
landslide_precip7d_df['days-to-landslide'] = count_down

# Add the normalized precipitation
accum_norm = []
for i, data in landslide_precip7d_df.groupby("landslide_id"):
    res = data['gpm_7day_accum_mm']/data['gpm_7day_accum_mm'].mean()
    accum_norm.append(res.max())

landslide_df['gpm_7day_accum_norm'] = accum_norm

In [ ]:
landslide_precip7d_df.info()

In [ ]:
landslide_df.info()

### Add the IMERGE 30min resolution precipitation to the landslide_df

#### First, read the IMERGE 30min data

In [ ]:
imerge_30min = get_imerge_hires(gpm_hires_files, glc_state_gt2015)

#### Loop over the newly created landslide_df
- Add the sum and the max

In [ ]:
imerge_7day_sum = []
imerge_7day_max = []
imerge_7day_accum = []
imerge_7day_date = []
imerge_7day_id = []

for i, ls_date in enumerate(landslide_df.index):

    glc_date = pd.date_range(ls_date, periods=7,
                             freq='-1D').strftime('%Y%m%d').to_list()

    # selecting rows based on condition
    rslt_df = imerge_30min[(imerge_30min['id'] == landslide_df['landslide_id'][i]) &
                           imerge_30min['yyyymmdd'].isin(glc_date)]
    # first calculate the 7 day precipitation stats
    if rslt_df.shape[0] > 0:
        imerge_7day_sum.append(rslt_df['precipitation'].sum())
        imerge_7day_max.append(rslt_df['precipitation'].max())
        # append the 7 day daily values
        imerge_7day_accum.extend(rslt_df.precipitation.resample('D').sum())
        imerge_7day_date.extend(glc_date)
        imerge_7day_id.extend([landslide_df['landslide_id'][i]]*7)
    else:
        imerge_7day_sum.append(np.nan)
        imerge_7day_max.append(np.nan)
        imerge_7day_accum.extend([np.nan]*7)
        imerge_7day_date.extend(glc_date)
        imerge_7day_id.extend([landslide_df['landslide_id'][i]]*7)

landslide_df['imerge_7day_accum_mm'] = imerge_7day_sum
landslide_df['imerge_7day_max_mm'] = imerge_7day_max
landslide_df.index.name = 'date'

landslide_precip7d_df['imerge_precip_mm'] = imerge_7day_accum

In [ ]:
landslide_precip7d_df

In [ ]:
# calculate the number of landslides per year
landslide_count = landslide_precip7d_df.groupby(landslide_precip7d_df.index.year)['smap_sm'].nunique().values.tolist()
print(landslide_count)
# year = np.arange(len(landslide_count))+2015
year = []
for i, df in landslide_df.groupby(landslide_df.index.year):
    year.append(df.index.year[0])

colors = ['dodgerblue',  'orange', 'firebrick', 'forestgreen', 'k', 'purple']
year_count = []
for i, yr in enumerate(year):
    year_count.append(str(yr)+' ('+str(landslide_count[i])+')')
    
landslide_dict = dict(zip(year_count, colors))
color_dict = dict(zip(year, colors))

print(landslide_dict)
print(color_dict)

### Plot the cumulative precipitation going back 7 days

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5), sharex=True, sharey=True)

for id, df in landslide_precip7d_df.groupby('landslide_id'):
    result = df['gpm_7day_accum_mm']/df['gpm_7day_accum_mm'].mean()
    if result.max() > 1.1:
        ax.plot(df['days-to-landslide'], result,
                color=color_dict[result.index.year[0]])

# Add legend
leg = ax.legend(handlelength=0, handletextpad=0, fancybox=True,
                labels=landslide_dict, labelcolor=color_dict.values(),
                facecolor='white')
for item in leg.legendHandles:
    item.set_visible(False)

# ax.invert_xaxis()
ax.axvline(x=7, ymin=0, color='mediumorchid', linestyle='dashed', linewidth=5)

ax.set(xlabel='Days leading up to rainfall-landslide event',
       ylabel='Normalized Precipitation',
       title=('GPM Cumultive Daily Rainfall over '
              + state+'\nNormalized to the 7-day mean'))
ax.text(7.1, 0, 'Landslide\nTriggered',
        color='mediumorchid', fontsize='medium')

# output the figure
output_fig = os.path.join(et.io.HOME,
                             'earth-analytics',
                             'capstone-landslides-soilmoisture', 'plots',
                             'gpm_precip7d_normalized_timeseries_'+state+'.png')

cache_override = False or GLOBAL_CACHE_OVERRIDE
if not os.path.exists(output_fig) or cache_override:
    plt.savefig(output_fig)

### Plot Precipitation vs SMAP Soil Moisture

In [ ]:
category = dict(zip(np.unique(landslide_df['landslide_category']), colors))
print(category)

trigger = dict(zip(np.unique(landslide_df['landslide_trigger']), colors))
print(trigger)

#### Plot by Landslide Category

In [ ]:
# Create a scatter plot
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(ncols=2, nrows=2, 
                                             figsize=(12, 12))

# GPM daily precip vs SMAP & ESA
for id, df in landslide_df.groupby("landslide_id"):
    for cat in category:
        if df['landslide_category'][0] == cat:
            ax1.scatter(df['smap_sm'], df['gpm_7day_accum_mm'],
                        edgecolor='k', s=50.0,
                        color=category[cat])
            ax2.scatter(df['esa_sm_percent'], df['gpm_7day_accum_mm'],
                        edgecolor='k', s=50.0,
                        color=category[cat])

# calculate the linear trend
idx = ( np.isfinite(landslide_df['smap_sm']) 
        & np.isfinite(landslide_df['gpm_7day_accum_mm'])
       )
# defining the variables
x = landslide_df['smap_sm'][idx].tolist()
y = landslide_df['gpm_7day_accum_mm'][idx].tolist()
# Calculate the OLR Bisector fit
intercept, slope, var = leastsq(x, y, method=3)
trend_mean_text = 'slope: '+str('{:.2f}'.format(slope)) + \
      ', y: '+str('{:.2f}'.format(intercept))

# plot the linear trend
ax1.plot(landslide_df['smap_sm'],
         intercept + slope*landslide_df['smap_sm'],
         color='crimson',
         linewidth=1.0)
ax1.text(0.15, 110, trend_mean_text)

ax1.set(xlabel='SMAP Soil moisture [$cm^3 cm^{-3}$]',
        ylabel='GPM 7day Precipitation Accummulation [mm]')
#ax1.set_ylim([-5, 120])

ax2.set(xlabel='ESA CCI Soil Moisture Saturation [%]',
        ylabel='GPM 7day Precipitation Accummulation [mm]')
#ax2.set_ylim([-5, 100])

# Add legend
leg = ax1.legend(handlelength=0, handletextpad=0, fancybox=True,
                 labels=category, labelcolor=colors, facecolor='white')
for item in leg.legendHandles:
    item.set_visible(False)

# add an overall caption
fig.suptitle(
    'Precipitaton vs Soil Moisture for 2015-2020 '+state+' Landslides'
)

# IMERGE 30min precip vs SMAP & ESA
for id, df in landslide_df.groupby("landslide_id"):
    for cat in category:
        if df['landslide_category'][0] == cat:
            ax3.scatter(df['smap_sm'], df['imerge_7day_accum_mm'],
                        edgecolor='k', s=50.0,
                        color=category[cat])
            ax4.scatter(df['esa_sm_percent'], df['imerge_7day_accum_mm'],
                        edgecolor='k', s=50.0,
                        color=category[cat])
ax3.set(xlabel='SMAP Soil moisture [$cm^3 cm^{-3}$]',
        ylabel='IMERGE 7day Precipitation Accummulation [mm]')
#ax3.set_ylim([-5, 120])

ax4.set(xlabel='ESA CCI Soil Moisture Saturation [%]',
        ylabel='IMERGE 7day Precipitation Accummulation [mm]')

# calculate the linear trend
idx = ( np.isfinite(landslide_df['smap_sm']) 
        & np.isfinite(landslide_df['imerge_7day_accum_mm'])
       )
# defining the variables
x = landslide_df['smap_sm'][idx].tolist()
y = landslide_df['imerge_7day_accum_mm'][idx].tolist()
# Calculate the OLR Bisector fit
intercept, slope, var = leastsq(x, y, method=3)
trend_mean_text = 'slope: '+str('{:.2f}'.format(slope)) + \
      ', y: '+str('{:.2f}'.format(intercept))

# plot the linear trend
ax3.plot(landslide_df['smap_sm'],
         intercept + slope*landslide_df['smap_sm'],
         color='crimson',
         linewidth=1.0)
ax3.text(0.25, 110, trend_mean_text)



# Add legend
leg = ax3.legend(handlelength=0, handletextpad=0, fancybox=True,
                 labels=category, labelcolor=colors, facecolor='white')
for item in leg.legendHandles:
    item.set_visible(False)

    
plt.tight_layout()

# output the figure
output_fig = os.path.join(et.io.HOME,
                          'earth-analytics',
                          'capstone-landslides-soilmoisture', 'plots',
                          'sm_VS_precip_2015_2020_category_'+state+'.png')

cache_override = False or GLOBAL_CACHE_OVERRIDE
if not os.path.exists(output_fig) or cache_override:
    plt.savefig(output_fig)

#### Plot by Landslide Trigger

### Scatter plots - Total precipitation vs maximum precipitation intensity

In [ ]:
landslide_precip7d_df

In [ ]:
# Create a scatter plot
fig, ax = plt.subplots(figsize=(8,8))

# GPM daily Pre3cip vs SMAP & ESA
xdata = []
ydata = []
for id, df in landslide_precip7d_df.groupby("landslide_id"):
    if df['gpm_precip_mm'].max() < 100.:
        ax.scatter((df['smap_sm'].max()-df['smap_sm'].min()), df['gpm_precip_mm'].max(), 
                        color='dodgerblue', edgecolor='k', s=60.0)
        xdata.append(df['smap_sm'].max()-df['smap_sm'].min())
        ydata.append(df['gpm_precip_mm'].max())
    

# Calculate the OLR Bisector fit
intercept, slope, var = leastsq(xdata, ydata, method=5)

trend_mean_text = 'slope: '+str('{:.2f}'.format(slope)) + \
      ', y: '+str('{:.2f}'.format(intercept))

# plot the linear trend
ax.plot(np.array(xdata),
         intercept + slope*np.array(xdata),
         color='crimson',
         linewidth=2.0)
ax.text(0.02, 50, trend_mean_text);

# Add titles
ax.set(xlabel='SMAP Soil Moisture 7day Difference [$cm^3 cm^{-3}$]',
        ylabel='GPM Precipitation Intensity [mm]');

# add an overall caption
fig.suptitle(
    'Precipitaton vs Soil Moisture\n2015-2020 '+state+' Landslides'
);

#output the figure
output_fig = os.path.join(et.io.HOME,
            'earth-analytics',
            'capstone-landslides-soilmoisture', 'plots',
            'smdiff_VS_precipintensity_2015_2020_'+state+'.png')

cache_override = False or GLOBAL_CACHE_OVERRIDE
if not os.path.exists(output_fig) or cache_override:
    plt.savefig(output_fig);

### Look at outliers by creating box-whisker plots

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 5))

red_sqr = dict(markerfacecolor='r', marker='s')

# Data may contain NaN - these have to be removed for boxplot() to work
filtered_data = landslide_df['smap_sm'][~np.isnan(landslide_df['smap_sm'])]

axs[0].boxplot(filtered_data, flierprops=red_sqr,
              whiskerprops = dict(linestyle='-',linewidth=2., color='steelblue'),
              boxprops = dict(linestyle='-',linewidth=2., color='steelblue'),
              medianprops = dict(linestyle='-',linewidth=3., color='orange'),
              capprops = dict(linestyle='-',linewidth=2., color='steelblue')
              )
axs[0].set(title='SMAP',
          ylabel='$cm^3 cm^{-3}$')

filtered_data = landslide_df['gpm_7day_accum_mm'][~np.isnan(landslide_df['gpm_7day_accum_mm'])]
axs[1].boxplot(filtered_data, flierprops=red_sqr,
              whiskerprops = dict(linestyle='-',linewidth=2., color='steelblue'),
              boxprops = dict(linestyle='-',linewidth=2., color='steelblue'),
              medianprops = dict(linestyle='-',linewidth=3., color='orange'),
              capprops = dict(linestyle='-',linewidth=2., color='steelblue')
              )
axs[1].set(title='GPM',
          ylabel='Accumulated Precip [mm]')

plt.suptitle(state+' 2015-2020 Landslide Events')

plt.tight_layout()

output_fig = os.path.join(et.io.HOME,
            'earth-analytics',
            'capstone-landslides-soilmoisture', 'plots',
            'sm_precip_2015_2020_boxplot_'+state+'.png')

#output the figure
cache_override = False or GLOBAL_CACHE_OVERRIDE
if not os.path.exists(output_fig) or cache_override:
    plt.savefig(output_fig);

In [ ]:
# fly = box['fliers'][0]
# outliers = fly.get_ydata()
# outliers